In [ ]:
from google.colab import drive, auth
drive.mount('/content/gdrive')
auth.authenticate_user()

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/JuncoFilms/

/content/gdrive/MyDrive/JuncoFilms


In [ ]:
!git clone https://github.com/coqui-ai/TTS

fatal: destination path 'TTS' already exists and is not an empty directory.


In [ ]:
%cd TTS

/content/gdrive/MyDrive/JuncoFilms/TTS


In [ ]:
!pip install -e .

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [ ]:
output_path = "/content/gdrive/MyDrive/JuncoFilms/"


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="transcript.txt", path=os.path.join(output_path, "EminemDataset/")
)

In [ ]:
%cd ..

/content/gdrive/MyDrive/JuncoFilms


In [ ]:
filename = "EminemDataset/transcript.txt"
with open(filename, "r", encoding="utf-8") as f:
    row_num = 0
    for line in f:
        print(line)

wav3| Will the real Slim Shady please stand up?

wav5| Will the real Slim Shady please stand up?

wav8| He didn't just say what I think he did, did he?

wav11| And if I'm lucky, you might just give it a little kiss. And that's the message that we deliver to little kids and expect them not to know what a woman's.

wav12| Of course they're gonna know what in the course is by the time they hit fourth grade. They got the Discovery Channel Don't they and we ain't nothing but mammals

wav18| Cause I'm Slim Shady, yes I'm the real Shady All you other Slim Shadys are just imitating So won't the real Slim Shady please stand up?

wav20| Please stand up. Will Smith don't got a cuss in his raps to sell records. Well, I do so

wav22| You too, you think I give a damn about a Grammy?

wav24| Let alone Stan Lee But Slim, what if you win? Wouldn't it be weird? Why? So you guys can just lie to get me here? So you can sit me here next to Britney Spears?

wav25| Christina Aguilera better switch me chairs 

In [ ]:
audio_config = VitsAudioConfig(
    sample_rate=22050,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    mel_fmin=50,
    mel_fmax=10000,
)

In [ ]:
character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890",
    punctuations=" !,.?-",
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [ ]:
config = VitsConfig(
    audio=audio_config,
    characters=character_config,
    run_name="vits_eminem",
    batch_size=16,
    eval_batch_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=0,
    run_eval=False,
    test_delay_epochs=0,
    epochs=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    save_best_after=1000,
    save_checkpoints=True,
    save_all_best=True,
    mixed_precision=True,
    max_text_len=250,  # change this if you have a larger VRAM than 16GB
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

In [ ]:
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:50
 | > mel_fmax:10000
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "eminem"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = f"/content/gdrive/MyDrive/JuncoFilms/EminemDataset/wavs/{cols[0].strip()}.wav"
            text = cols[1].strip().replace('\n','n').replace('%', 'percent')

            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [ ]:
train_samples, eval_samples = load_tts_samples(
dataset_config,
eval_split=False,
eval_split_size=0.045454545454545456,
formatter=formatter)

 | > Found 22 files in /content/gdrive/MyDrive/JuncoFilms/EminemDataset


In [ ]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/gdrive/MyDrive/JuncoFilms/vits_eminem-September-27-2023_09+35PM-0000000

 > Model has 83047468 parameters


In [ ]:
trainer.fit()


 > EPOCH: 0/1000
 --> /content/gdrive/MyDrive/JuncoFilms/vits_eminem-September-27-2023_09+35PM-0000000
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-09-27 21:35:43) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 22
 | > Preprocessing samples
 | > Max text length: 207
 | > Min text length: 41
 | > Avg text length: 111.36363636363636
 | 
 | > Max audio length: 190115.0
 | > Min audio length: 46261.0
 | > Avg audio length: 116739.59090909091
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
i guess there's a slim shady in all of us.
cause i'm slim shady, yes i'm the real shady all you other slim shadys are just imitating so won't the real slim shady please stand up? [!] Character "'" not found in the vocabulary. Discarding it.

 [!] Character "'" not found in the vocabulary. Discarding it.



   --> TIME: 2023-09-27 21:35:45 -- STEP: 0/2 -- GLOBAL_STEP: 0
     | > loss_disc: 6.098871231079102  (6.098871231079102)
     | > loss_disc_real_0: 1.0301625728607178  (1.0301625728607178)
     | > loss_disc_real_1: 1.0344454050064087  (1.0344454050064087)
     | > loss_disc_real_2: 1.017329216003418  (1.017329216003418)
     | > loss_disc_real_3: 1.0289796590805054  (1.0289796590805054)
     | > loss_disc_real_4: 1.0129642486572266  (1.0129642486572266)
     | > loss_disc_real_5: 0.9739564061164856  (0.9739564061164856)
     | > loss_0: 6.098871231079102  (6.098871231079102)
     | > grad_norm_0: 0  (0)
     | > loss_gen: 6.098204612731934  (6.098204612731934)
     | > loss_kl: 152.40829467773438  (152.40829467773438)
     | > loss_feat: 0.36705437302589417  (0.36705437302589417)
     | > loss_mel: 104.6734848022461  (104.6734848022461)
     | > loss_duration: 1.6462924480438232  (1.6462924480438232)
     | > amp_scaler: 32768.0  (32768.0)
     | > loss_1: 265.1933288574219  (265.1

 | > Synthesizing test sentences.
it took me quite a long time to develop a voice, and now that i have it i'm not going to be silent.
 [!] Character "'" not found in the vocabulary. Discarding it.


/content/gdrive/MyDrive/JuncoFilms/TTS/TTS/tts/models/vits.py:1455: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  test_figures["{}-alignment".format(idx)] = plot_alignment(alignment.T, output_fig=False)

  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0043010711669921875 (+0)
     | > avg_step_time: 1.6641881465911865 (+0)
     | > avg_loss_disc: 6.098668098449707 (+0)
     | > avg_loss_disc_real_0: 1.0301800966262817 (+0)
     | > avg_loss_disc_real_1: 1.0344618558883667 (+0)
     | > avg_loss_disc_real_2: 1.0171147584915161 (+0)
     | > avg_loss_disc_real_3: 1.0290738344192505 (+0)
     | > avg_loss_disc_real_4: 1.0128772258758545 (+0)
     | > avg_loss_disc_real_5: 0

...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0031943321228027344 (+0.00021028518676757812)
     | > avg_step_time: 0.5399975776672363 (+0.10163283348083496)
     | > avg_loss_disc: 2.35927414894104 (-0.10814881324768066)
     | > avg_loss_disc_real_0: 0.07329946756362915 (-0.05971387028694153)
     | > avg_loss_disc_real_1: 0.21340374648571014 (-0.08221618831157684)
     | > avg_loss_disc_real_2: 0.22283487021923065 (-0.010347351431846619)
     | > avg_loss_disc_real_3: 0.1891641616821289 (-0.022642120718955994)
     | > avg_loss_disc_real_4: 0.14020320773124695 (-0.2152785360813141)
     | > avg_loss_disc_real_5: 0.22310827672481537 (+0.04208393394947052)
     | > avg_loss_0: 2.35927414894104 (-0.10814881324768066)
     | > avg_grad_norm_0: tensor(23.5330, device='cuda:0') (+tensor(7.7937, device='cuda:0'))
     | > avg_loss_gen: 2.688539981842041 (+0.5135304927825928)
     | > avg_loss_kl: 1.4725288152694702 (+0.05419135093688965)
     | > avg_loss_feat: 3.462702751159668 (+0.

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0043637752532958984 (+0.001169443130493164)
     | > avg_step_time: 0.6868243217468262 (+0.14682674407958984)
     | > avg_loss_disc: 2.037123203277588 (-0.32215094566345215)
     | > avg_loss_disc_real_0: 0.08372563123703003 (+0.010426163673400879)
     | > avg_loss_disc_real_1: 0.24635589122772217 (+0.032952144742012024)
     | > avg_loss_disc_real_2: 0.23380926251411438 (+0.010974392294883728)
     | > avg_loss_disc_real_3: 0.12986208498477936 (-0.05930207669734955)
     | > avg_loss_disc_real_4: 0.23817509412765503 (+0.09797188639640808)
     | > avg_loss_disc_real_5: 0.14435696601867676 (-0.07875131070613861)
     | > avg_loss_0: 2.037123203277588 (-0.32215094566345215)
     | > avg_grad_norm_0: tensor(11.6489, device='cuda:0') (tensor(-11.8841, device='cuda:0'))
     | > avg_loss_gen: 2.30068039894104 (-0.387859582901001)
     | > avg_loss_kl: 1.540985107421875 (+0.06845629215240479)
     | > avg_loss_feat: 4.017899513244629 (+0

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.004152059555053711 (-0.0002117156982421875)
     | > avg_step_time: 0.48649120330810547 (-0.2003331184387207)
     | > avg_loss_disc: 2.2087786197662354 (+0.17165541648864746)
     | > avg_loss_disc_real_0: 0.16028434038162231 (+0.07655870914459229)
     | > avg_loss_disc_real_1: 0.26405519247055054 (+0.01769930124282837)
     | > avg_loss_disc_real_2: 0.2259228527545929 (-0.007886409759521484)
     | > avg_loss_disc_real_3: 0.25421515107154846 (+0.1243530660867691)
     | > avg_loss_disc_real_4: 0.26161396503448486 (+0.023438870906829834)
     | > avg_loss_disc_real_5: 0.1797681301832199 (+0.03541116416454315)
     | > avg_loss_0: 2.2087786197662354 (+0.17165541648864746)
     | > avg_grad_norm_0: tensor(16.4874, device='cuda:0') (+tensor(4.8385, device='cuda:0'))
     | > avg_loss_gen: 2.2693490982055664 (-0.03133130073547363)
     | > avg_loss_kl: 1.6844377517700195 (+0.14345264434814453)
     | > avg_loss_feat: 3.6320743560791016 

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%cd JuncoFilms

/content/gdrive/MyDrive/JuncoFilms


In [ ]:
import glob, os
output_path = "."
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
ckpts, configs

(['./vits_eminem-September-27-2023_09+35PM-0000000/best_model.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1472.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1740.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1750.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1794.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1802.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1852.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1874.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1910.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1938.pth',
  './vits_eminem-September-27-2023_09+35PM-0000000/best_model_1986.pth'],
 ['./vits_eminem-September-27-2023_09+35PM-0000000/config.json'])

In [ ]:
test_ckpt = ckpts[0]
test_config = configs[0]

In [ ]:
 !tts --text "My voice has been recreated so everybody can feel overwhelmed" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:50
 | > mel_fmax:10000
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: My voice has been recreated so everybody can feel overwhelmed
 > Text splitted to sentences.
['My voice has been recreated so everybody can feel overwhelmed']
 > Processing time: 2.738386869430542
 > Real-time factor: 0.879

In [ ]:
import IPython
IPython.display.Audio("out.wav", rate=22050)